## 1. GPU 확인

In [53]:
!nvidia-smi

Fri Jan  6 17:47:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 26%   34C    P5    17W / 125W |   3431MiB /  6144MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 2. Lib Import

In [123]:
# arrange an instance segmentation model for test
from sahi.utils.yolov5 import (
    download_yolov5s6_model,
)

# import required functions, classes
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from sahi.scripts.coco_error_analysis import analyse
from sahi.scripts.coco_evaluation import evaluate
from IPython.display import Image
from pathlib import Path
import json
import os
from pathlib import Path
from PIL import Image

## 3. Data & Model Path 선언

In [124]:
source_image_dir = "../resources/FLL_VAL/images/"
source_label_dir = "../resources/FLL_VAL/labels/"

# 960 imgsz로 훈련된 FLL Target
fll_model_221024_960_path = '../resources/models/221024_960/best.pt'

## 4. Model Load

In [125]:
fll_model_221024_960 = AutoDetectionModel.from_pretrained(
    model_type='yolov5',
    model_path=fll_model_221024_960_path,
    confidence_threshold=0.25,
    device="cuda:0",
    image_size=960
)

In [126]:
model = fll_model_221024_960
model_path = fll_model_221024_960_path

## 5. Gt Json 생성

In [127]:
def initial_extract(img_dir, label_dir, out_dir):
    if os.path.exists(os.path.join(out_dir, 'val.json')):
        os.remove(os.path.join(out_dir, 'val.json'))
    
    licenses = [
        {
            "name": "",
            "id": 0,
            "url": ""
        }
    ]

    info_ = [
        {
            "contributor": "",
            "date_created": "",
            "description": "",
            "url": "",
            "version": "",
            "year": ""
        }
    ]

    categories = [
        {
            "id": 0,
            "name": "Buoy",
            "supercategory": ""
        },
        {
            "id": 1,
            "name": "Boat",
            "supercategory": ""
        },
        {
            "id": 2,
            "name": "Channel Marker",
            "supercategory": ""
        },
        {
            "id": 3,
            "name": "Speed Warning Sign",
            "supercategory": ""
        }
    ]

    img_idx = 0
    annot_idx = 0

    imgs_list = []
    annots_list = []

    for label_file in sorted(os.listdir(label_dir)):
        label_file_ = os.path.join(label_dir, label_file)
        img_file_ = os.path.join(img_dir, f'{os.path.splitext(label_file)[0]}.jpg')
        img = Image.open(img_file_)
        image_w, image_h = img.size

        imgs_list.append({
            'id': img_idx,
            'width': image_w,
            'height': image_h,
            'file_name': f'{os.path.splitext(label_file)[0]}.jpg',
            "license": 0,
            "flickr_url": "",
            "coco_url": "",
            "date_captured": 0
        })

        with open(label_file_, 'r') as label_f:
            labels = label_f.readlines()

            for label in labels:
                cat, xc, yc, label_normalized_w, label_normalized_h = list(map(lambda x: int(x) if len(x) == 1 else float(x), label.split()))
                label_w, label_h = image_w * label_normalized_w, image_h * label_normalized_h
                xmin, ymin = (image_w * xc) - (label_w / 2), (image_h * yc) - (label_h / 2)
                
                xmin = 0 if xmin < 0 else xmin
                ymin = 0 if ymin < 0 else ymin

                annots_list.append({
                    'id': annot_idx,
                    'image_id': img_idx,
                    'category_id': cat,
                    'area': int(label_h * label_w),
                    'bbox': [
                        xmin,
                        ymin,
                        label_w,
                        label_h
                    ],
                    'iscrowd': 0,
                    'attributes': {
                        'type': '',
                        'occluded': False
                    },
                    'segmentation': []
                })

                annot_idx += 1

        img_idx += 1

    out_dict = {
        'licenses': licenses,
        'info': info_,
        'categories': categories,
        'images': imgs_list,
        'annotations': annots_list
    }
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)
    
    with open(os.path.join(out_dir, 'val.json'), 'w') as out_f:
        print(os.path.join(out_dir, 'val.json'))
        json.dump(out_dict, out_f)
        
    return os.path.join(out_dir, 'val.json')

In [128]:
# initial_extract(img_dir, label_dir, out_dir)
gt_json_path = initial_extract(source_image_dir, source_label_dir, str(Path(source_image_dir).parent))

../resources/FLL_VAL/val.json


## 6. Eval hyper-param Setting

In [129]:
INFERENCE_SETTING_TO_PARAMS = {
    "AVIKUS_FL": {
        "model_confidence_threshold": 0.25,
        "model_device": "cuda:0",
        "image_size": 960,
        "postprocess_type": "GREEDYNMM",
        "postprocess_match_metric": "IOS",
        "no_standard_prediction": False, # no FULL inference? 
        "no_sliced_prediction": False, # no Tiling? 
        "slice_size": 640, # Slice size when activate tiling
        "overlap_ratio": 0.0, # Overlap ratio when activate tiling
        "match_threshold": 0.5, # Merge match thresh when activate tiling
        "postprocess_class_agnostic": False,  # class agnostic when activate tiling
        "custom_slice_y_start": 200,  # Y start point when activate tiling
        "custom_slice_x_start": 640,
        "custom_slice_mode": 2
    },
}

MODEL_TYPE = "yolov5" # model type
MODEL_PATH = model_path # model path
MODEL_CONFIG_PATH = ""
EVAL_IMAGES_FOLDER_DIR = source_image_dir # source dir 
EVAL_DATASET_JSON_PATH = gt_json_path # gt json path
INFERENCE_SETTING = "AVIKUS_FL"
EXPORT_VISUAL = False
MAX_DETECTIONS = 300

setting_params = INFERENCE_SETTING_TO_PARAMS[INFERENCE_SETTING]

## 참고. Interact로 인퍼런스 확인

In [132]:
from sahi.utils.cv import Colors
import numpy as np
import copy

def visualize_object_predictions(
    image: np.array,
    object_prediction_list,
    rect_th: int = None,
    text_size: float = None,
    text_th: float = None,
    color: tuple = None,
):
    """
    Visualizes prediction category names, bounding boxes over the source image
    and exports it to output folder.
    Arguments:
        object_prediction_list: a list of prediction.ObjectPrediction
        rect_th: rectangle thickness
        text_size: size of the category name over box
        text_th: text thickness
        color: annotation color in the form: (0, 255, 0)
        output_dir: directory for resulting visualization to be exported
        file_name: exported file will be saved as: output_dir+file_name+".png"
        export_format: can be specified as 'jpg' or 'png'
    """
    # deepcopy image so that original is not altered
    image = copy.deepcopy(image)
    # select predefined classwise color palette if not specified
    if color is None:
        colors = Colors()
    else:
        colors = None
    # set rect_th for boxes
    rect_th = rect_th or max(round(sum(image.shape) / 2 * 0.001), 1)
    # set text_th for category names
    text_th = text_th or max(rect_th - 1, 1)
    # set text_size for category names
    text_size = text_size or rect_th / 3
    # add bbox and mask to image if present
    for object_prediction in object_prediction_list:
        # deepcopy object_prediction_list so that original is not altered
        object_prediction = object_prediction.deepcopy()

        bbox = object_prediction.bbox.to_voc_bbox()
        category_name = object_prediction.category.name
        score = object_prediction.score.value

        # set color
        if colors is not None:
            color = colors(object_prediction.category.id)
        # visualize masks if present
        if object_prediction.mask is not None:
            # deepcopy mask so that original is not altered
            mask = object_prediction.mask.bool_mask
            # draw mask
            rgb_mask = apply_color_mask(mask, color)
            image = cv2.addWeighted(image, 1, rgb_mask, 0.4, 0)
        # set bbox points
        p1, p2 = (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3]))
        # visualize boxes
        cv2.rectangle(
            image,
            p1,
            p2,
            color=color,
            thickness=rect_th
        )
        # arange bounding box text location
        label = f"{category_name} {score:.2f}"
        w, h = cv2.getTextSize(label, 0, fontScale=text_size, thickness=text_th)[0]  # label width, height
        outside = p1[1] - h - 3 >= 0  # label fits outside box
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        # add bounding box text
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(
            image,
            label,
            (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
            0,
            text_size,
            (255, 255, 255),
            thickness=text_th,
        )
        
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
    return image

In [140]:
from ipywidgets import interact
from sahi.slicing import slice_image
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

image_files = sorted(os.listdir(source_image_dir))

@interact(index=(0, len(image_files)-1),
          slice_size=(0, 640),
          overlap_ratio=(0, 0.5, 0.05),
          custom_slice_x_start=(0, 640),
          custom_slice_y_start=(0, 512),
          custom_slice_mode=(0,3),
          only_full_inference=(0,1),
          before_merge_view=(0,1))
def show_sample(index=0, slice_size=640, overlap_ratio=0.25,
                custom_slice_x_start=640, custom_slice_y_start=360, custom_slice_mode=2,
                only_full_inference=0, before_merge_view=0):
    image_file = image_files[index]
    image_path = os.path.join(source_image_dir, image_file)
    image = cv2.imread(image_path)
    
    if not only_full_inference:
        slice_result = slice_image(image_path, 
                                  slice_width=slice_size,
                                  slice_height=slice_size,
                                  overlap_height_ratio=overlap_ratio,
                                  overlap_width_ratio=overlap_ratio,
                                  custom_slice_x_start=custom_slice_x_start,
                                  custom_slice_y_start=custom_slice_y_start,
                                  custom_slice_mode=custom_slice_mode,
                                  verbose=1)

        for start_pixel in slice_result.starting_pixels:
            cv2.rectangle(image,
                          start_pixel,
                          [s1+s2 for s1, s2 in zip(start_pixel,[slice_size,slice_size])],
                          color=(255, 255, 0),
                          thickness=2)
        
        if before_merge_view:
            only_slice_result = get_sliced_prediction(image_path,
                                           model,
                                           slice_height=slice_size,
                                           slice_width=slice_size,
                                           postprocess_match_threshold=0.5,
                                           overlap_height_ratio=overlap_ratio,
                                           overlap_width_ratio=overlap_ratio,
                                           custom_slice_x_start=custom_slice_x_start,
                                           custom_slice_y_start=custom_slice_y_start,
                                           custom_slice_mode=custom_slice_mode,
                                           perform_standard_pred=False
                                          )
            only_standard_result = get_prediction(image_path, model)
        else:
            result = get_sliced_prediction(image_path,
                                       model,
                                       slice_height=slice_size,
                                       slice_width=slice_size,
                                       postprocess_match_threshold=0.5,
                                       overlap_height_ratio=overlap_ratio,
                                       overlap_width_ratio=overlap_ratio,
                                       custom_slice_x_start=custom_slice_x_start,
                                       custom_slice_y_start=custom_slice_y_start,
                                       custom_slice_mode=custom_slice_mode
                                      )
    else:
        result = get_prediction(image_path, model)
    
    if before_merge_view and not only_full_inference: 
        canvas = visualize_object_predictions(image, only_slice_result.object_prediction_list, color=(255,0,0))
        canvas = visualize_object_predictions(cv2.cvtColor(canvas, cv2.COLOR_RGB2BGR), only_standard_result.object_prediction_list, color=(0,0,255))
    else:
        canvas = visualize_object_predictions(image, result.object_prediction_list)
        
    plt.figure(figsize=(16,16))
    plt.imshow(canvas)
    plt.axis('off')
    plt.show()

interactive(children=(IntSlider(value=0, description='index', max=4896), IntSlider(value=640, description='sli…

In [162]:
im = np.random.randn(640,640,3)
size=(960,960)
im = im[..., :3] if im.ndim == 3 else cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)  # enforce 3ch input
s = im.shape[:2]  # HWC
g = max(size) / max(s)  # gain
shape1 = ([y * g for y in s])
print(shape1)

[960.0, 960.0]


In [160]:
def letterbox(im, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints
    shape = im.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better val mAP)
        r = min(r, 1.0)
    
    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    print(new_unpad)
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding
    elif scaleFill:  # stretch
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  # width, height ratios

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        im = cv2.resize(im, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return im, ratio, (dw, dh)

In [161]:
im, ratio, (dw, dh) = letterbox(im, new_shape=(960,960))
im.shape, ratio, (dw, dh)

(960, 960)


((960, 960, 3), (1.5, 1.5), (0.0, 0.0))

## 7. Execute SAHI 1slice Inference

In [117]:
setting_params

{'model_confidence_threshold': 0.25,
 'model_device': 'cuda:0',
 'image_size': 960,
 'postprocess_type': 'GREEDYNMM',
 'postprocess_match_metric': 'IOS',
 'no_standard_prediction': False,
 'no_sliced_prediction': False,
 'slice_size': 640,
 'overlap_ratio': 0.0,
 'match_threshold': 0.5,
 'postprocess_class_agnostic': False,
 'custom_slice_y_start': 200,
 'custom_slice_x_start': 640,
 'custom_slice_mode': 2}

In [118]:
result = predict(
    detection_model=model,
    model_type=MODEL_TYPE,
    model_path=MODEL_PATH,
    model_config_path=MODEL_CONFIG_PATH,
    model_confidence_threshold=setting_params["model_confidence_threshold"],
    model_device=setting_params["model_device"],
    model_category_mapping=None,
    model_category_remapping=None,
    source=EVAL_IMAGES_FOLDER_DIR,
    no_standard_prediction=setting_params["no_standard_prediction"],
    no_sliced_prediction=setting_params["no_sliced_prediction"],
    slice_height=setting_params["slice_size"],
    slice_width=setting_params["slice_size"],
    overlap_height_ratio=setting_params["overlap_ratio"],
    overlap_width_ratio=setting_params["overlap_ratio"],
    image_size=setting_params["image_size"],
    postprocess_type=setting_params["postprocess_type"],
    postprocess_match_metric=setting_params["postprocess_match_metric"],
    postprocess_match_threshold=setting_params["match_threshold"],
    postprocess_class_agnostic=setting_params["postprocess_class_agnostic"],
    novisual=not EXPORT_VISUAL,
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    project="runs/FLL_FULL_INFERENCE",
    name=INFERENCE_SETTING,
    visual_bbox_thickness=None,
    visual_text_size=None,
    visual_text_thickness=None,
    visual_export_format="png",
    verbose=2,
    return_dict=True,
    force_postprocess_type=False,
    custom_slice_mode=setting_params["custom_slice_mode"],
    custom_slice_x_start=setting_params["custom_slice_x_start"],
    custom_slice_y_start=setting_params["custom_slice_y_start"],
)

result_json_path = str(Path(result["export_dir"]) / "result.json")

indexing coco dataset annotations...


Performing inference on images:   0%|                                                                                                                       | 2/4897 [00:00<06:32, 12.46it/s]

Performing prediction on 1 number of slices.
Prediction time is: 60.68 ms
Performing prediction on 1 number of slices.
Prediction time is: 59.65 ms
Performing prediction on 1 number of slices.


Performing inference on images:   0%|                                                                                                                       | 4/4897 [00:00<06:19, 12.91it/s]

Prediction time is: 59.14 ms
Performing prediction on 1 number of slices.
Prediction time is: 51.86 ms
Performing prediction on 1 number of slices.
Prediction time is: 53.06 ms
Performing prediction on 1 number of slices.


...

Prediction time is: 51.14 ms
Performing prediction on 1 number of slices.
Prediction time is: 50.74 ms
Performing prediction on 1 number of slices.
Prediction time is: 50.82 ms
Performing prediction on 1 number of slices.


Performing inference on images: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 4894/4897 [05:52<00:00, 14.16it/s]

Prediction time is: 50.56 ms
Performing prediction on 1 number of slices.
Prediction time is: 52.21 ms
Performing prediction on 1 number of slices.
Prediction time is: 53.27 ms
Performing prediction on 1 number of slices.


Performing inference on images: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4897/4897 [05:53<00:00, 13.87it/s]


Prediction time is: 53.62 ms
Performing prediction on 1 number of slices.
Prediction time is: 51.95 ms
Prediction results are successfully exported to runs/FLL_FULL_INFERENCE/AVIKUS_FL15
Model loaded in 2.384185791015625e-07 seconds.
Slicing performed in 7.481736183166504 seconds.
Prediction performed in 254.74270915985107 seconds.


## 8. mAP evaluation 

In [119]:
evaluate_dict = evaluate(
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    result_json_path=result_json_path,
    classwise=True,
    max_detections=MAX_DETECTIONS,
    return_dict=True,
)

loading annotations into memory...
Done (t=0.21s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.64s).
Accumulating evaluation results...
DONE (t=0.21s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.199
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=300 ] = 0.407
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=300 ] = 0.165
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=300 ] = 0.297
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=300 ] = 0.677
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=300 ] = 0.918
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=300 ] = 0.123
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=300 ] = 0.357
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [120]:
with open(result_json_path, 'r') as f:
    sahi_json = json.load(f)

In [121]:
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

@interact(index=(0, len(image_files)-1))
def show_sample(index=0):
    infos = [ data for data in sahi_json if data['image_id'] == index ]
    image_path = os.path.join(source_image_dir, image_files[index])
    image = cv2.imread(image_path)    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    for info in infos:
        bbox = list(map(int, info['bbox']))
        category_name =  info['category_name']
        conf = info['score']
        label = f"{category_name} {conf:.2f}"
        
        cv2.rectangle(image,
                      bbox[:2],
                      [ bbox1 + bbox2 for bbox1, bbox2 in zip(bbox[:2], bbox[2:])],
                      color=(255, 0, 0),
                      thickness=2)
        cv2.putText(
            image,
            label,  
            [bbox[0], bbox[1]-10],
            0,
            1,
            (255, 255, 255),
            thickness=3,
        )
        
    plt.figure(figsize=(16,16))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

interactive(children=(IntSlider(value=0, description='index', max=4896), Output()), _dom_classes=('widget-inte…

## 11. Export DSL Preproc 1Slice prediction result.json

In [82]:
dsl_preproc_1slice_json = "../../aiboat/aiboat/APP/NAS/deepstream-services-library/examples/python/result_FL221024_0.25_preproc_1slice.json"

In [83]:
evaluate_dict = evaluate(
    dataset_json_path=EVAL_DATASET_JSON_PATH,
    result_json_path=dsl_preproc_1slice_json,
    classwise=True,
    max_detections=MAX_DETECTIONS,
    return_dict=True,
)

loading annotations into memory...
Done (t=0.16s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.65s).
Accumulating evaluation results...
DONE (t=0.21s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.265
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=300 ] = 0.515
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=300 ] = 0.237
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=300 ] = 0.450
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=300 ] = 0.740
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=300 ] = 0.757
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=300 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=300 ] = 0.423
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

## 12. Visualize json file

In [84]:
with open(dsl_preproc_1slice_json, 'r') as f:
    dsl_json = json.load(f)

In [85]:
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

@interact(index=(0, len(image_files)-1))
def show_sample(index=0):
    infos = [ data for data in dsl_json if data['image_id'] == index ]
    image_path = os.path.join(source_image_dir, image_files[index])
    image = cv2.imread(image_path)    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    for info in infos:
        bbox = list(map(int, info['bbox']))
        category_name =  info['category_name']
        conf = info['score']
        label = f"{category_name} {conf:.2f}"
        
        cv2.rectangle(image,
                      bbox[:2],
                      [ bbox1 + bbox2 for bbox1, bbox2 in zip(bbox[:2], bbox[2:])],
                      color=(255, 0, 0),
                      thickness=2)
        cv2.putText(
            image,
            label,  
            [bbox[0], bbox[1]-10],
            0,
            1,
            (255, 255, 255),
            thickness=3,
        )
        
    plt.figure(figsize=(16,16))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

interactive(children=(IntSlider(value=0, description='index', max=4896), Output()), _dom_classes=('widget-inte…

## 13. Compare SAHI and DSL

In [46]:
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import copy

@interact(index=(0, len(image_files)-1))
def show_sample(index=0):
    sahi_infos = [ data for data in sahi_json if data['image_id'] == index ]
    dsl_infos = [ data for data in dsl_json if data['image_id'] == index ]
    
    image_path = os.path.join(source_image_dir, image_files[index])
    image = cv2.imread(image_path)    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image2 = copy.deepcopy(image)
    
    for info in sahi_infos:
        bbox = list(map(int, info['bbox']))
        category_name =  info['category_name']
        conf = info['score']
        label = f"{category_name} {conf:.2f}"
        
        cv2.rectangle(image,
                      bbox[:2],
                      [ bbox1 + bbox2 for bbox1, bbox2 in zip(bbox[:2], bbox[2:])],
                      color=(255, 0, 0),
                      thickness=2)
        cv2.putText(
            image,
            label,  
            [bbox[0], bbox[1]-10],
            0,
            1,
            (255, 255, 255),
            thickness=3,
        )
    
    for info in dsl_infos:
        bbox = list(map(int, info['bbox']))
        category_name =  info['category_name']
        conf = info['score']
        label = f"{category_name} {conf:.2f}"
        
        cv2.rectangle(image2,
                      bbox[:2],
                      [ bbox1 + bbox2 for bbox1, bbox2 in zip(bbox[:2], bbox[2:])],
                      color=(255, 0, 0),
                      thickness=2)
        cv2.putText(
            image2,
            label,  
            [bbox[0], bbox[1]-10],
            0,
            1,
            (255, 255, 255),
            thickness=3,
        )
    
        
    plt.figure(figsize=(16,16))
    plt.subplot(1,2,1)
    plt.imshow(image)
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.imshow(image2)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

interactive(children=(IntSlider(value=0, description='index', max=4896), Output()), _dom_classes=('widget-inte…

In [ ]:
# 1182
# 1120
# 1595
# 1601
# 1765 ####

In [ ]:
# SAHI,DSL에서 생겨나는 버그에 대해서 확인하기

## SAHI Postprocess Debug

In [122]:
from ipywidgets import interact
from sahi.slicing import slice_image
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

image_files = sorted(os.listdir(source_image_dir))

@interact(index=(0, len(image_files)-1),
          slice_size=(0, 640),
          overlap_ratio=(0, 0.5, 0.05),
          postprocess_match_threshold=(0,1,0.1),
          custom_slice_x_start=(0, 640),
          custom_slice_y_start=(0, 512),
          custom_slice_mode=(0,3),
          only_full_inference=(0,1))
def show_sample(index=0, slice_size=640, overlap_ratio=0.25, postprocess_match_threshold=0.5,
                custom_slice_x_start=640, custom_slice_y_start=200, custom_slice_mode=2,
                only_full_inference=0):
    image_file = image_files[index]
    image_path = os.path.join(source_image_dir, image_file)
    image = cv2.imread(image_path)
    
    if not only_full_inference:
        slice_result = slice_image(image_path, 
                                  slice_width=slice_size,
                                  slice_height=slice_size,
                                  overlap_height_ratio=overlap_ratio,
                                  overlap_width_ratio=overlap_ratio,
                                  custom_slice_x_start=custom_slice_x_start,
                                  custom_slice_y_start=custom_slice_y_start,
                                  custom_slice_mode=custom_slice_mode,
                                  verbose=1)

        for start_pixel in slice_result.starting_pixels:
            cv2.rectangle(image,
                          start_pixel,
                          [s1+s2 for s1, s2 in zip(start_pixel,[slice_size,slice_size])],
                          color=(255, 255, 0),
                          thickness=2)

        result = get_sliced_prediction(image_path,
                                       model,
                                       slice_height=slice_size,
                                       slice_width=slice_size,
                                       overlap_height_ratio=overlap_ratio,
                                       overlap_width_ratio=overlap_ratio,
                                       custom_slice_x_start=custom_slice_x_start,
                                       custom_slice_y_start=custom_slice_y_start,
                                       postprocess_match_threshold=postprocess_match_threshold,
                                       custom_slice_mode=custom_slice_mode
                                      )
    else:
        result = get_prediction(image_path, model)
    
    canvas = visualize_object_predictions(image, result.object_prediction_list)
    plt.figure(figsize=(16,16))
    plt.imshow(canvas)
    plt.axis('off')
    plt.show()

interactive(children=(IntSlider(value=0, description='index', max=4896), IntSlider(value=640, description='sli…